In [59]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit
from sklearn import   linear_model, metrics
import math
from sklearn.linear_model import SGDClassifier as SGD
import numpy as np
import matplotlib.pyplot as plt

In [60]:
train_directory = "data/train.csv"
train = pd.read_csv(train_directory, header=0 )
train['y'] = [1 if y == 'yes' else 0 for y in train['y']]

In [61]:
X = train.drop(['y'], axis=1)
y = train['y']

In [62]:
dtypes = []
cat_features = []
for idx, col in enumerate(X.columns):
    dtype = X[col].dtype
    dtypes.append({'column': col, 'dtype':dtype})
    if dtype == 'O':
        cat_features.append(idx)

pd.DataFrame(dtypes)

,column,dtype
0,age,int64
1,job,object
2,marital,object
3,education,object
4,default,object
5,housing,object
6,loan,object
7,contact,object
8,month,object
9,day_of_week,object


In [63]:
categorial_features_rate = 100*len(cat_features)/len(dtypes)
print(f'Доля категориальных признаков в данных: {int(categorial_features_rate)}%')
print("Самое время использовать катбуст!")
import catboost

Доля категориальных признаков в данных: 50%
Самое время использовать катбуст!


In [64]:
cat_features

[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [65]:
y.value_counts()

0    2860
1     362
Name: y, dtype: int64

Видим, что выборка несбалансирована: проставим веса для классов в соотвествии с пропорцией классов

In [87]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=123)
POSITIVE_SAMPLE_WEIGHT = 10

score = []

for train_index, test_index in sss.split(X, y):
    catboost_model = catboost.CatBoostClassifier(
        cat_features=cat_features,
        iterations=100,
        thread_count=8,
        loss_function='CrossEntropy'
    )

    X_train = X.values[train_index]
    y_train = y.values[train_index]
    
    X_test = X.values[test_index]
    y_test = y.values[test_index]
    
    sample_weight = [POSITIVE_SAMPLE_WEIGHT if item else 1 for item in y_train]
    
    catboost_model.fit(
        X_train ,
        y_train,
        logging_level='Silent',
        sample_weight=sample_weight
    )

    catboost_predict = catboost_model.predict(X_test)
    
    score.append(
        metrics.roc_auc_score(y_test, catboost_predict, sample_weight=[POSITIVE_SAMPLE_WEIGHT if item  else 1 for item in y_test])
    )
score = np.array(score)
print(f'Mean ROC AUC: {score.mean()}')

Mean ROC AUC: 0.8577661431064574


Старая метрика: 0.5432140700483092

In [89]:
old_metric = 0.5432140700483092
new_metric = score.mean()

improvement_rate = ((new_metric-old_metric)/old_metric)*100

print(f'Метрика улучшилась на {improvement_rate:.3}%!')

Метрика улучшилась на 57.9%!


In [90]:
print('Важности признаков модели')
for col_name, importance in zip(X.columns, catboost_model.feature_importances_):
    print(f'{col_name}:{importance}')

Важности признаков модели
age:0.7826656484748102
job:0.5905750895936358
marital:0.9076979522827102
education:2.950543767549
default:2.263373058765679
housing:1.7106466647751701
loan:2.7794231404250365
contact:0.885681434305319
month:7.9731492476957655
day_of_week:3.203547188809914
duration:54.37868885159839
campaign:2.8575516119706665
pdays:0.2585134600040851
previous:0.3141462404644101
poutcome:2.085965019895715
emp.var.rate:2.0091181375983735
cons.price.idx:0.8712229570640152
cons.conf.idx:0.2140769281129982
euribor3m:6.290338277190442
nr.employed:6.673075323423873


In [91]:
train['nr.employed'].value_counts()

5228.1    1268
5099.1     656
5191.0     599
5195.8     300
5076.2     119
5017.5      77
4963.6      73
4991.6      71
5008.7      45
5023.5      13
5176.3       1
Name: nr.employed, dtype: int64

In [94]:
metrics.recall_score(y_test, catboost_predict, sample_weight=[10 if item  else 1 for item in y_test])

0.8333333333333334

In [95]:
metrics.precision_score(y_test, catboost_predict, sample_weight=[10 if item  else 1 for item in y_test])

0.8875739644970414

## Ответ к заданию: 0.86

<b>напишите 3-5 действий по итогу оценки качества, которые Вы бы сделали для улучшения качества модели, если бы Вам дали на работу еще 5 рабочих дней. </b>


Вместо тупого использования со стандартными параметрами на всех признаках можно подобрать только значашие признаки и попытаться обяснить зависимость, подобрать опитимальные параметры модели

Можно попробовать найти кореллирующие признаки и объединить группу кореллирующих признаков в один

Помимо этого я заметил, что некоторые числовые признаки (пример с nr.employed выше, и некоторые другие) представляют собой скорее категориальные, т.к. имеют совсем небольшое количество различающихся значений

Можно попробовать сделать что-то, связанное с этим

В списке важности признаков есть строчка duration:54.37868885159839, т.е. признак который имеет очень большую важность по сравнению с остальными, это можно использовать для улучшения модели
